In [1]:
import sys
import os
import numpy as np
import time
from imp import reload

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")

new directory is: C:\Users\Luke\Desktop\Barracuda\AutomatedCE


In [2]:
def get_float(msg:str, failed=False):
    """
    :param msg: message to display
    :param failed: flag if user has failed the question requirments
    :return : float number
    """
    if failed:
        print("Please enter a number")
    resp = input(msg+ " ")
    if resp.replace('.','').isnumeric():
        return float(resp)
    else:
        return get_float(msg,failed=True)
    
def get_int(msg:str, failed=False):
    """
    :param msg: message to display
    :param failed: flag if user has failed the question requirments
    :return : float number
    """
    if failed:
        print("Please enter a number")
    resp = input(msg+ " ")
    if resp.isnumeric():
        return int(resp)
    else:
        return get_float(msg,failed=True)
def get_yes_no(msg:str, failed=False):
    """
    Asks user for Y/N response
    :return : True for Yes, False for No
    """
    if failed:
        print("Please enter y or n as you answer")
    resp = input(msg+ " ")
    if resp.lower()=='n':
        return False
    elif resp.lower()=='y':
        return True
    else:
        return get_yes_no(msg,failed=True)

##  Load the CE System with a Config File

Select one of the predesigned configurations (TE300, Eclipse Ti, Simulation). See an example config under "/config/test-system.cfg". 

Change the variable CONFIG_PATH to the absolute path where the config file resides. 

Be sure to make the string a raw string by prepending *'r'*. **Example:** r"C:\path\to\config\"*

In [3]:
CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()
ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


 Change System Config setting if thorlabs is needed. 


[] 0
{}
-0.1 300.0 <class 'float'>
[] 0
{}
-0.1 300.0 <class 'float'>
['offset', '25', 'default', '20', 'min_z', '0', 'max_z', '25', 'scale', '1', 'invert', '1'] 6
{'offset': '25', 'default': '20', 'min_z': '0', 'max_z': '25', 'scale': '1', 'invert': '1'}
0.0 25.0 <class 'float'>
{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x00000280862DB908>, 'xy_stage': <L2.XYControl.PycromanagerXY object at 0x00000280862DB848>, 'objective': <L2.ZControl.PycromanagerZ object at 0x00000280862DBD88>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x00000280862E1188>, 'inlet_z': <L2.ZControl.ArduinoZ object at 0x00000280862DBE48>, 'filter_wheel': None, 'excitation_shutter': None, 'camera': <L2.CameraControl.PycromanagerControl object at 0x00000280862E1548>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x00000280862E1948>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x00000280862E1748>}


# Make a New Template


### Step 1. Enter in basic information
Also if you would want to home the stage to a special position, now would be the time. If the home position is changed partway through, you will have to restart the template making process. 

Run the following cell if you wish to restart (it will remove all well positions)

In [18]:
header_info = "Basic Template for Eclipse using Native Home position" # String that describes the template
dimensions = [130, 150] # Width and Height of the Template
new_template = AutomatedControl.TemplateMaker()


### Step 2. Get the outer limits

Move the stage to the lower left corner, then run the next cell

In [15]:
lower_left_corner = ce_system.xy_stage.read_xy()

Move the stage to the upper right corner, then run the next cell

In [16]:
upper_right_coner = ce_system.xy_stage.read_xy()

## Step 3. Add a Single Well (repeatable)
Move the capillary to the center of the well you want to add then run the next cell. 

In [19]:
new_well_name = input("What would you like to call this well? ")
new_well_width = get_float("What is the well width in mm? ")
resp = input("Position the well directly under the capillary. Press enter to continue: ")
new_well_center = ce_system.xy_stage.read_xy()
new_template.add_well(new_well_name, new_well_center, new_well_width, 'Circle')
print(f"Well {new_well_name} added.")

What would you like to call this well?S1
What is the well width in mm? 1.4


### Step 4. Add an Array of Wells (repeatable)
Add an array  of wells specified by two corners of the well array. 


In [20]:
name = input("What is the prefix for these wells? ")
size = get_float("What is the radius of these wells? ")
rows = get_int("Enter the number of rows: ")
cols = get_int("Enter the number of columns: ")
resp = input("Move the well array to the center of the lower_left well")
xy1 = ce_system.xy_stage.read_xy()
resp = input("Move the well array to the center of the upper right well")
xy2 = ce_system.xy_stage.read_xy()
new_template.add_array(name, xy1, xy2, rows, cols, size, 'circle')


What is the prefix for these wells? S
What is the radius of these wells?  2
Enter the number of rows:  5
Enter the number of columns:  8
Move the well array to the center of the lower_left well
Move the well array to the center of the upper right well


### Step 5. Add a Ledge

Ledges contain the height information so that the program knows when to raise the capillary while moving the XY stage. For now, enter a safe height the capillary can move to for the entire stage

In [24]:
safe_height = get_float("Please enter a safe height for capillary in mm: ")
new_template.add_ledge('Safe Height', (0,0), 500, 'circle', safe_height)

Please enter a safe height for capillary in mm:  24


### Step 6. Save and Name the template

In [31]:
import os
file_name = input("Please enter the name for this template (it will be saved to the var folder): ")
file_path = os.path.relpath(f'./var/{file_name}.txt')
x1,y1 = lower_left_corner
x2,y2 = upper_right_coner
new_template.add_dimension(x1, y1, x2, y2)
new_template.save_to_file(file_path)

Please enter the name for this template (it will be saved to the var folder): test


True